# MODEL TRAINING  USING THE TF OBJECT DETECTION API

This notebook is based on modular [shell scripts](./scripts) for convenience. The scripts comprise minimal lines of code (LOC), on average, no more than 10 LOC for each, making them easy to read, learn, and adapt for future applications.

**!!!Caveat :** 

Some scripts can be conveniently executed directly from this notebook, and some need to be executed in terminal to grant permissions. I recommend running each script from the terminal to see STDOUT clearly. If run in terminal, use the [module-training](./) directory as the working directory.

***This is an academic project. Interested developers are invited to freely contribute issues, questions, improvements, and discussions.***

---

## 1. CHECK SUBMODULES

[`check-submodules.sh`](./scripts/) makes sure that the [LabelImg](https://github.com/tzutalin/labelImg) and [TF models](https://github.com/tensorflow/models) submodule repositories are initialized correctly.

In [12]:
!./scripts/check-submodules.sh

-- checking submodules


## 2. CREATE VIRTUAL ENVIRONMENT

[`venv.sh`](./scripts/) creates a virtual environment and installs essential [apt](./resources/apt.txt) and [pip](./resources/pip.txt) dependencies into the environment. As such, the script should be executed in terminal to grant install permissions [( see dependencies.sh )](./scripts/). By default, the name of the virtual environment is **tfod-venv**. This name can, of cause, be changed. From working directory run `./scripts/venv.sh`

---

In [1]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2

In [2]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /home/everett/.cache/torch/hub/master.zip


  0%|          | 0.00/755k [00:00<?, ?B/s]

YOLOv5 🚀 2021-9-2 torch 1.9.0+cu111 CUDA:0 (GeForce RTX 2070 SUPER, 7974.3125MB)



  0%|          | 0.00/14.1M [00:00<?, ?B/s]

Fusing layers... 
Model Summary: 224 layers, 7266973 parameters, 0 gradients
Adding AutoShape... 
